# Quantum Machine Learning Introduction

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/umitkacar/Awesome-AI-Resources/blob/main/notebooks/intermediate/quantum_ml_intro.ipynb)

Learn the basics of Quantum Machine Learning with hands-on examples using Qiskit and PennyLane.

## 1. Install Quantum Libraries

In [ ]:
# Install quantum computing libraries
!pip install -q qiskit qiskit-machine-learning
!pip install -q pennylane
!pip install -q matplotlib numpy scikit-learn

## 2. Quantum Basics: Creating a Bell State

In [ ]:
from qiskit import QuantumCircuit, execute, Aer
from qiskit.visualization import plot_histogram, plot_bloch_multivector
import matplotlib.pyplot as plt

# Create a quantum circuit with 2 qubits
qc = QuantumCircuit(2, 2)

# Create Bell state: |00⟩ + |11⟩
qc.h(0)  # Hadamard gate on qubit 0
qc.cx(0, 1)  # CNOT gate: control=0, target=1

# Measure both qubits
qc.measure([0, 1], [0, 1])

# Visualize the circuit
qc.draw('mpl')

In [ ]:
# Execute the circuit
backend = Aer.get_backend('qasm_simulator')
job = execute(qc, backend, shots=1000)
result = job.result()
counts = result.get_counts(qc)

# Plot results
plot_histogram(counts)

## 3. Variational Quantum Classifier (VQC)

In [ ]:
import numpy as np
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Generate dataset
X, y = make_moons(n_samples=100, noise=0.1, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Visualize data
plt.scatter(X_train[:, 0], X_train[:, 1], c=y_train, cmap='viridis')
plt.title('Moon Dataset')
plt.show()

In [ ]:
from qiskit.circuit.library import ZZFeatureMap, RealAmplitudes
from qiskit_machine_learning.algorithms import VQC
from qiskit.algorithms.optimizers import COBYLA

# Create feature map
feature_map = ZZFeatureMap(feature_dimension=2, reps=2)

# Create ansatz (variational circuit)
ansatz = RealAmplitudes(num_qubits=2, reps=3)

# Create VQC
vqc = VQC(
    feature_map=feature_map,
    ansatz=ansatz,
    optimizer=COBYLA(maxiter=100),
    quantum_instance=Aer.get_backend('qasm_simulator')
)

# Visualize the quantum circuit
print("Feature Map:")
feature_map.decompose().draw('mpl', style='iqx')

In [ ]:
# Train the quantum classifier
print("Training Quantum Classifier...")
vqc.fit(X_train_scaled, y_train)

# Make predictions
y_pred = vqc.predict(X_test_scaled)

# Calculate accuracy
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, y_pred)
print(f"\nAccuracy: {accuracy:.2%}")

## 4. Quantum Neural Network with PennyLane

In [ ]:
import pennylane as qml
import torch
import torch.nn as nn

# Create quantum device
n_qubits = 4
dev = qml.device('default.qubit', wires=n_qubits)

@qml.qnode(dev)
def quantum_circuit(inputs, weights):
    # Encode inputs
    for i in range(n_qubits):
        qml.RY(inputs[i % len(inputs)], wires=i)
    
    # Variational layers
    for layer in range(2):
        for i in range(n_qubits):
            qml.RY(weights[layer, i, 0], wires=i)
            qml.RZ(weights[layer, i, 1], wires=i)
        
        # Entanglement
        for i in range(n_qubits - 1):
            qml.CNOT(wires=[i, i + 1])
    
    # Measure expectation values
    return [qml.expval(qml.PauliZ(i)) for i in range(n_qubits)]

In [ ]:
# Hybrid Classical-Quantum Model
class HybridModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.classical_layer = nn.Linear(2, 4)
        self.quantum_weights = nn.Parameter(torch.randn(2, 4, 2) * 0.1)
        self.output_layer = nn.Linear(4, 1)
    
    def forward(self, x):
        # Classical preprocessing
        x = torch.tanh(self.classical_layer(x))
        
        # Quantum processing
        batch_size = x.shape[0]
        quantum_out = []
        
        for i in range(batch_size):
            q_out = quantum_circuit(x[i], self.quantum_weights)
            quantum_out.append(q_out)
        
        quantum_out = torch.tensor(quantum_out)
        
        # Classical postprocessing
        output = torch.sigmoid(self.output_layer(quantum_out))
        return output.squeeze()

In [ ]:
# Convert data to PyTorch tensors
X_train_torch = torch.FloatTensor(X_train_scaled)
y_train_torch = torch.FloatTensor(y_train)
X_test_torch = torch.FloatTensor(X_test_scaled)
y_test_torch = torch.FloatTensor(y_test)

# Create and train hybrid model
model = HybridModel()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = nn.BCELoss()

# Training loop
epochs = 50
for epoch in range(epochs):
    optimizer.zero_grad()
    outputs = model(X_train_torch)
    loss = criterion(outputs, y_train_torch)
    loss.backward()
    optimizer.step()
    
    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')

## 5. Quantum Kernel Methods

In [ ]:
from qiskit_machine_learning.kernels import QuantumKernel
from sklearn.svm import SVC

# Create quantum kernel
quantum_kernel = QuantumKernel(
    feature_map=feature_map,
    quantum_instance=Aer.get_backend('statevector_simulator')
)

# Compute kernel matrix
print("Computing quantum kernel matrix...")
kernel_matrix_train = quantum_kernel.evaluate(x_vec=X_train_scaled)
kernel_matrix_test = quantum_kernel.evaluate(x_vec=X_test_scaled, y_vec=X_train_scaled)

# Use quantum kernel with classical SVM
qsvm = SVC(kernel='precomputed')
qsvm.fit(kernel_matrix_train, y_train)

# Predictions
y_pred_qsvm = qsvm.predict(kernel_matrix_test)
accuracy_qsvm = accuracy_score(y_test, y_pred_qsvm)
print(f"\nQuantum SVM Accuracy: {accuracy_qsvm:.2%}")

## 6. Compare Classical vs Quantum Performance

In [ ]:
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier

# Classical SVM
classical_svm = SVC(kernel='rbf')
classical_svm.fit(X_train_scaled, y_train)
classical_accuracy = classical_svm.score(X_test_scaled, y_test)

# Classical Neural Network
mlp = MLPClassifier(hidden_layer_sizes=(10, 10), max_iter=1000)
mlp.fit(X_train_scaled, y_train)
mlp_accuracy = mlp.score(X_test_scaled, y_test)

# Results comparison
results = {
    'Classical SVM': classical_accuracy,
    'Classical NN': mlp_accuracy,
    'Quantum VQC': accuracy,
    'Quantum Kernel SVM': accuracy_qsvm
}

# Visualize results
plt.figure(figsize=(10, 6))
plt.bar(results.keys(), results.values())
plt.ylabel('Accuracy')
plt.title('Classical vs Quantum Model Performance')
plt.ylim(0, 1)
for i, (name, acc) in enumerate(results.items()):
    plt.text(i, acc + 0.01, f'{acc:.2%}', ha='center')
plt.show()

## 7. Quantum Advantage Analysis

In [ ]:
# Dataset size analysis
dataset_sizes = [20, 50, 100, 200, 500]
quantum_times = []
classical_times = []

import time

for size in dataset_sizes:
    # Generate data
    X, y = make_moons(n_samples=size, noise=0.1)
    X_scaled = scaler.fit_transform(X)
    
    # Time quantum kernel
    start = time.time()
    kernel_matrix = quantum_kernel.evaluate(x_vec=X_scaled[:20])  # Limit for demo
    quantum_time = time.time() - start
    quantum_times.append(quantum_time)
    
    # Time classical SVM
    start = time.time()
    svm = SVC(kernel='rbf')
    svm.fit(X_scaled[:20], y[:20])
    classical_time = time.time() - start
    classical_times.append(classical_time)

# Plot scaling
plt.figure(figsize=(10, 6))
plt.plot(dataset_sizes, quantum_times, 'o-', label='Quantum', linewidth=2)
plt.plot(dataset_sizes, classical_times, 's-', label='Classical', linewidth=2)
plt.xlabel('Dataset Size')
plt.ylabel('Time (seconds)')
plt.title('Computational Time Scaling')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

## 🎯 Exercises

1. **Modify the VQC**: Try different feature maps (like `PauliFeatureMap`) and ansatzes
2. **Quantum Data Encoding**: Implement amplitude encoding for higher dimensional data
3. **QAOA Implementation**: Use QAOA to solve a simple optimization problem
4. **Noise Simulation**: Add noise to the quantum circuits and analyze impact

## 📚 Further Reading

- [Quantum ML Basics](https://github.com/umitkacar/Awesome-AI-Resources/blob/main/QuantumML/quantum-ml-basics.md)
- [Quantum Algorithms](https://github.com/umitkacar/Awesome-AI-Resources/blob/main/QuantumML/quantum-algorithms.md)
- [QML Frameworks](https://github.com/umitkacar/Awesome-AI-Resources/blob/main/QuantumML/qml-frameworks.md)
- [Qiskit Textbook](https://qiskit.org/textbook/)
- [PennyLane Demos](https://pennylane.ai/qml/demonstrations.html)